In [1]:
import cv2
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from keras.preprocessing.image import load_img #import image need this
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Activation, Dropout, Flatten, Dense,Input
from keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.applications import VGG16
from keras.applications import VGG19
from keras.applications import ResNet50
from keras.applications import MobileNetV2
from keras.callbacks import ModelCheckpoint
from keras.utils.vis_utils import plot_model
from keras.models import Model
from keras import optimizers
import tensorflow as  tf
import keras.backend as K
import os
import random
import scipy
from sklearn.metrics import classification_report, confusion_matrix,roc_curve,roc_auc_score,auc
from keras.layers.advanced_activations import LeakyReLU

Using TensorFlow backend.


In [2]:
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [3]:
def history_plot(results): 
    # list all data in history
    print(results.history.keys())
    # summarize history for accuracy
    plt.figure(1)
    plt.plot(results.history['acc'])
    plt.plot(results.history['val_acc'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'validation'], loc='upper left')
    plt.savefig('Accuracy',dpi=300)
    plt.show()

    # summarize history for loss
    plt.figure(2)
    plt.plot(results.history["loss"], label="loss")
    plt.plot(results.history["val_loss"], label="val_loss")
    plt.plot(np.argmin(results.history["val_loss"]), 
             np.min(results.history["val_loss"]), 
             marker="x", color="r", label="best model")
    plt.xlabel("Epochs")
    plt.ylabel("log_loss")
    plt.legend();
    plt.savefig('Loss',dpi=300)
    plt.show()

In [4]:
#flow_from_directory目錄需要分3類
train_image = './pneu_dataset/train'
valid_image = './pneu_dataset/validation'
test_image = './pneu_dataset/test'

In [5]:

# Setting the default value
# dimensions of our images.
img_width, img_height = 224 , 224
batch_size = 32
epochs = 300
seed = 42
class_mode = 'categorical'
date = '2019-06-16_BL_cell'

In [6]:
#ImageDataGenerator
# used to rescale the pixel values from [0, 255] to [0, 1] interval
datagen = ImageDataGenerator(rescale=1./255)
#                              rotation_range = 90,  #圖片隨機轉動
#                              width_shift_range = 0.2, #圖片水平偏移
#                              height_shift_range = 0.2, #圖片垂直偏移
#                              zoom_range = 0.3)


# automagically retrieve images and their classes for train and validation sets
train_generator = datagen.flow_from_directory(
       train_image,
        target_size = (img_width, img_height),
        batch_size = batch_size,
        class_mode = class_mode,
        seed = seed)

validation_generator = datagen.flow_from_directory(
        valid_image,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode = class_mode,
        shuffle = False ,
        seed = seed)

# test_generator = datagen.flow_from_directory(
#         test_image,
#         target_size = (img_width, img_height),
#         batch_size = 1,
#         class_mode = None,
#         shuffle = False,
#         seed = seed
#         )

Found 19689 images belonging to 3 classes.
Found 6471 images belonging to 3 classes.


In [7]:
# ## Model way transfer learning
# initial_model =VGG16(include_top=False,weights='imagenet')
# input = Input(shape=(128,128,3),name = 'image_input')
# output_vgg16_conv = initial_model(input)

# x = Flatten()(output_vgg16_conv)
# x = Dense(4096, activation='relu', name='fc1')(x)
# x = Dense(4096, activation='relu', name='fc2')(x)
# x = Dense(3, activation='softmax', name='predictions')(x)

# model = Model(input= input ,output = x)

model = Sequential()
model.add(Convolution2D(32, 3, 3, input_shape=(img_width, img_height,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Convolution2D(32, 3, 3))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Convolution2D(64, 3, 3))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(3))
model.add(Activation('softmax'))

# Transfer learning
# model = Sequential()
# for layer in VGG16().layers:
#     model.add(layer)
# model.layers.pop()
# model.add(Dense(3, activation='softmax'))

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


/home/cheyu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:14: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), input_shape=(224, 224,...)`
  
/home/cheyu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:18: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3))`
/home/cheyu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:22: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3))`


In [ ]:
## John model
# model = Sequential()
# model.add(Convolution2D(64, 3, 3, input_shape=(img_width, img_height,3)))
# model.add(LeakyReLU(alpha=0.2))
# model.add(MaxPooling2D(pool_size=(2, 2)))

# model.add(Convolution2D(128, 3, 3))
# model.add(LeakyReLU(alpha=0.2))
# model.add(MaxPooling2D(pool_size=(2, 2)))

# model.add(Convolution2D(256, 3, 3))
# model.add(LeakyReLU(alpha=0.2))
# model.add(MaxPooling2D(pool_size=(2, 2)))

# model.add(Flatten())
# model.add(Dense(128))
# model.add(LeakyReLU(alpha=0.2))
# model.add(Dropout(0.5))
# model.add(Dense(3))
# model.add(Activation('softmax'))


In [8]:
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [9]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 222, 222, 32)      896       
_________________________________________________________________
activation_1 (Activation)    (None, 222, 222, 32)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 111, 111, 32)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 109, 109, 32)      9248      
_________________________________________________________________
activation_2 (Activation)    (None, 109, 109, 32)      0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 54, 54, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 52, 52, 64)        18496     
__________

In [ ]:
# 設定checkpoint & callbacks
from keras.callbacks import EarlyStopping
from keras.callbacks import ReduceLROnPlateau
from keras.callbacks import ModelCheckpoint
def callbacks(name):
    save_dir = "checkpoint"
    #     tl.files.exists_or_mkdir(save_dir)
    path=save_dir+'/model_check_'+name+'.h5'
    callbacks = [
        EarlyStopping(patience=35, verbose=1),
        ReduceLROnPlateau(factor=0.1, patience=10, min_lr=0.000001, verbose=1),
        ModelCheckpoint(path, verbose=1, save_best_only=True, save_weights_only=False)
#         ModelCheckpoint(path,monitor='val_acc', verbose=1,
#                         save_best_only=True, save_weights_only=False)
    ]
    return callbacks

In [ ]:
num_of_train_samples  = train_generator.classes.shape[0]
num_of_test_samples = validation_generator.classes.shape[0]

In [ ]:
history = model.fit_generator(
        train_generator,
        steps_per_epoch= batch_size,
        validation_data=validation_generator,
        validation_steps= batch_size,
        epochs = epochs,
        callbacks=callbacks(date+'_fit'))

In [ ]:
history_plot(history)

In [ ]:
# Load CheckPoint
# from keras.models import load_model
# model= load_model('./checkpoint/model_check_2019-03-14_1_fit.h5')

In [ ]:
# evaluate the model
model.evaluate_generator(generator=validation_generator,
#                          steps=validation_generator.classes.shape[0]//32 +1,
                         steps=validation_generator.classes.shape[0],
                         verbose=1)

In [ ]:

# predict the output
Y_pred = model.predict_generator(validation_generator,
                                 steps=num_of_test_samples//32 +1,
                                 verbose=1)

In [ ]:
## Use auc to show the performance
# auc = roc_auc_score(validation_generator.classes,Y_pred)
# print("AUC: {}".format(auc))
# fpr, tpr, thresholds = roc_curve(validation_generator.classes, Y_pred)

In [ ]:
## Plot ROC curve
# plt.figure(3)
# plt.plot([0, 1], [0, 1], 'k--')
# plt.plot(fpr,tpr,label='AUC = %0.2f' % auc)
# plt.xlabel('False Positive Rate')
# plt.ylabel('True Positive Rate')
# plt.title('ROC Curve')
# # plt.show()
# plt.savefig("roc_curve.png",dpi=300)

In [10]:
# Load CheckPoint
from keras.models import load_model 
base_model= load_model('./checkpoint/model_check_2019-06-16_BL_cell_fit.h5')

Instructions for updating:
Use tf.cast instead.


In [11]:
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_formats = {'png', 'retina'}
plt.rcParams['figure.figsize'] = 8,8

from scipy.ndimage.interpolation import zoom
from keras.preprocessing.image import load_img, img_to_array
from keras.applications.vgg16 import VGG16, preprocess_input, decode_predictions
from keras.preprocessing import image
from tensorflow.python.framework import ops
import keras.backend as K
import tensorflow as tf
import numpy as np
import keras
import cv2
import os
import gradcamutils

In [12]:
# most import value
def decode_predictions_cust(preds, class_custom, top=3):
    results = []

    for pred in preds:
        top_indices = pred.argsort()[-top:][::-1]
        result = [tuple(class_custom[i][0])+(class_custom[i][1],) + (pred[i]*100,)\
                  for i in top_indices] 
        results.append(result)
    return results

In [13]:
count = 0
class_name=[('0','Lung Opacity'),('1','Normal'),('2','Not Normal')]
label = [] # label Lung Opacity, Normal, Not Normal
# path of img 
valid_lung_op_path='./pneu_dataset/validation/Lung Opacity/'
valid_normal_path='./pneu_dataset/validation/Normal/'
valid_not_normal_path='./pneu_dataset/validation/Not Normal/'
paths = [valid_lung_op_path,valid_normal_path,valid_not_normal_path]
# list of each img
valid_path = []
# dir 
for path in paths:
    for file in  os.listdir(path):
        if path == './pneu_dataset/validation/Lung Opacity/':
            valid_path.append(path+file)
            label.append('Lung Opacity')
        elif path == './pneu_dataset/validation/Normal/':
            valid_path.append(path+file)
            label.append('Normal')
        elif path == './pneu_dataset/validation/Not Normal/':
            valid_path.append(path+file)
            label.append('Not Normal')
        else :
            label.append('None')

In [15]:
# use grad cam & ++ 
for path in valid_path:
    count+=1
    orig_img = np.array(load_img(path,target_size=(224,224)),dtype=np.uint8)
    img = np.array(load_img(path,target_size=(224,224)),dtype=np.float64)
    img = np.expand_dims(img,axis=0)
    img = preprocess_input(img)
    predictions = base_model.predict(img)
    top_n = 3
    top = decode_predictions_cust(predictions,class_name ,top=top_n)[0]
#     top = decode_predictions(predictions, top=top_n)[0]
    cls = np.argsort(predictions[0])[-top_n:][::-1]
    
    gradcam=gradcamutils.grad_cam(base_model,img,layer_name='conv2d_3')
    gradcamplus=gradcamutils.grad_cam_plus(base_model,img,layer_name='conv2d_3')
    print(path)
    print("class activation map for:",top[0])
    fig, ax = plt.subplots(nrows=1,ncols=3)
    plt.subplot(131)
    plt.imshow(orig_img)
    plt.title("input image \n"+'('+str(label[count])+')')
    plt.subplot(132)
    plt.imshow(orig_img)
    plt.imshow(gradcam,alpha=0.4,cmap="jet")
    plt.title("Grad-CAM \n"+str(top[0][1])+'\n'+str(top[0][2])[:6])
    
    plt.subplot(133)
    plt.imshow(orig_img)
    plt.imshow(gradcamplus,alpha=0.4,cmap="jet")
    plt.title("Grad-CAM++ \n"+str(top[0][1])+'\n'+str(top[0][2])[:6])
#     plt.savefig('./grad_cam_image/bloodcell/BL_gram&++_'+str(count)+'.png',dpi=300)
    plt.show()
    break

ValueError: Error when checking input: expected conv2d_1_input to have shape (128, 128, 3) but got array with shape (224, 224, 3)

In [ ]:
gradcamplus

# grad cam only another method

In [ ]:
def normalize(x):
    # utility function to normalize a tensor by its L2 norm
    return x / (K.sqrt(K.mean(K.square(x))) + 1e-5)


def load_image(path,target_size=(256, 256)):
    img_path = path
    img = image.load_img(img_path, target_size=target_size)
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    return x


def register_gradient():
    if "GuidedBackProp" not in ops._gradient_registry._registry:
        @ops.RegisterGradient("GuidedBackProp")
        def _GuidedBackProp(op, grad):
            dtype = op.inputs[0].dtype
            return grad * tf.cast(grad > 0., dtype) * \
                tf.cast(op.inputs[0] > 0., dtype)


def compile_saliency_function(model, activation_layer='block5_pool'):
    input_img = model.input
    layer_dict = dict([(layer.name, layer) for layer in model.layers[1:]])
    layer_output = layer_dict[activation_layer].output
    max_output = K.max(layer_output, axis=3)
    saliency = K.gradients(K.sum(max_output), input_img)[0]
    return K.function([input_img, K.learning_phase()], [saliency])


def modify_backprop(model, name):
    g = tf.get_default_graph()
    with g.gradient_override_map({'Relu': name}):

        # get layers that have an activation
        layer_dict = [layer for layer in model.layers[1:]
                      if hasattr(layer, 'activation')]

        # replace relu activation
        for layer in layer_dict:
            if layer.activation == keras.activations.relu:
                layer.activation = tf.nn.relu

        # re-instanciate a new model
        new_model = VGG16(weights='imagenet')
    return new_model


def deprocess_image(x):
    '''
    Same normalization as in:
    https://github.com/fchollet/keras/blob/master/examples/conv_filter_visualization.py
    '''
    if np.ndim(x) > 3:
        x = np.squeeze(x)
    # normalize tensor: center on 0., ensure std is 0.1
    x -= x.mean()
    x /= (x.std() + 1e-5)
    x *= 0.1

    # clip to [0, 1]
    x += 0.5
    x = np.clip(x, 0, 1)

    # convert to RGB array
    x *= 255
    if K.image_dim_ordering() == 'th':
        x = x.transpose((1, 2, 0))
    x = np.clip(x, 0, 255).astype('uint8')
    return x


def grad_cam(model, x, category_index, layer_name):
    """
    Args:
       model: model
       x: image input
       category_index: category index
       layer_name: last convolution layer name
    """
    # get category loss
    class_output = model.output[:, category_index]

    # layer output
    convolution_output = model.get_layer(layer_name).output
    # get gradients
    grads = K.gradients(class_output, convolution_output)[0]
    # get convolution output and gradients for input
    gradient_function = K.function([model.input], [convolution_output, grads])

    output, grads_val = gradient_function([x])
    output, grads_val = output[0], grads_val[0]

    # avg
    weights = np.mean(grads_val, axis=(0, 1))
    cam = np.dot(output, weights)

    # create heat map
    cam = cv2.resize(cam, (x.shape[1], x.shape[2]), cv2.INTER_LINEAR)
    cam = np.maximum(cam, 0)
    heatmap = cam / np.max(cam)

    # Return to BGR [0..255] from the preprocessed image
    image_rgb = x[0, :]
    image_rgb -= np.min(image_rgb)
    image_rgb = np.minimum(image_rgb, 255)

    cam = cv2.applyColorMap(np.uint8(255 * heatmap), cv2.COLORMAP_JET)
    cam = np.float32(cam) + np.float32(image_rgb)
    cam = 255 * cam / np.max(cam)
    return np.uint8(cam), heatmap

In [ ]:
# pic_cam_folder = './grad_cam_image/bloodcell/'


# arr_images = []

# for i, path in enumerate(valid_path):
#     img = load_image(path,(128,128))

#     predictions = base_model.predict(img)
#     top_1 = decode_predictions_cust(predictions,class_name)[0][0]
#     print('Predicted class:')
#     print('%s (%s) with probability %.2f' % (top_1[1], top_1[0], top_1[2]))

#     predicted_class = np.argmax(predictions)
#     cam_image, heat_map = grad_cam(base_model, img, predicted_class, "conv2d_3")
    
#     img_file = image.load_img(path)
#     img_file = image.img_to_array(img_file)

#     # guided grad_cam img
#     register_gradient()
#     guided_model = modify_backprop(base_model, 'GuidedBackProp')
#     saliency_fn = compile_saliency_function(guided_model)

#     saliency = saliency_fn([img, 0])
#     grad_cam_img = saliency[0] * heat_map[..., np.newaxis]
#  # save img

#     cam_image = cv2.resize(cam_image, (img_file.shape[1], img_file.shape[0]), cv2.INTER_LINEAR)
#     cv2.putText(cam_image,str(top_1[1]), (20, 20), cv2.FONT_HERSHEY_COMPLEX_SMALL, 1,(0, 0, 255))
#     cv2.putText(cam_image,str(top_1[2]), (20, 50), cv2.FONT_HERSHEY_COMPLEX_SMALL, 1,(0, 0, 255))

#     grad_cam_img = deprocess_image(grad_cam_img)
#     grad_cam_img = cv2.resize(grad_cam_img, (img_file.shape[1], img_file.shape[0]), cv2.INTER_LINEAR)
#     cv2.putText(grad_cam_img,str(top_1[1]), (20, 20), cv2.FONT_HERSHEY_COMPLEX_SMALL, 1,(0, 0, 255))
#     cv2.putText(grad_cam_img,str(top_1[2]), (20, 50), cv2.FONT_HERSHEY_COMPLEX_SMALL, 1,(0, 0, 255))

#     cam_image = cam_image.astype('float32')
#     grad_cam_img = grad_cam_img.astype('float32')
#     im_h = cv2.hconcat([img_file, cam_image, grad_cam_img])
#     cv2.imwrite(pic_cam_folder +top_1[1] +'_'+str(i)+'.jpg', im_h)